In [1]:
import os
%pwd

'c:\\Mlops-projects\\project-1\\datascienceproject\\research'

In [2]:
os.chdir("../")

In [12]:
os.environ["MLFLOW_TRACKING_URI"]="https://dagshub.com/YagantiAshok177/datascienceproject.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"]="YagantiAshok177"
os.environ["MLFLOW_TRACKING_PASSWORD"]="Ashok123@"

In [4]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class ModelEvalutionConfig():
    root_dir: Path
    test_datapath: Path
    model_filepath: Path
    metrics_file_name: Path
    target_column: str
    all_params: dict
    mlflow_uri: str

In [5]:
from src.datascience.utils.common import read_yaml,create_directories,save_json
from src.datascience.constants import *

In [6]:
class ConfigurationManager():
    def __init__(self,config_filepath=CONFIG_FILE_PATH,params_filepath=PARAMS_FILE_PATH,schema_filepath=SCHEMA_FILE_PATH):
        self.config=read_yaml(config_filepath)
        self.params=read_yaml(params_filepath)
        self.schema=read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])
    
    def get_model_evaluation_data(self) -> ModelEvalutionConfig:
        config=self.config.model_evaluation
        params=self.params.ElasticNet
        schema=self.schema.TARGET_COLUMN
        
        create_directories([config.root_dir])

        model_evaluation_data=ModelEvalutionConfig(
            root_dir=config.root_dir,
            test_datapath=config.test_datapath,
            model_filepath=config.model_filepath,
            metrics_file_name=config.metrics_file_name,
            target_column=schema.name,
            all_params=params,
            mlflow_uri=os.environ["MLFLOW_TRACKING_URI"]

          )
        
        return model_evaluation_data

In [7]:
import pandas as pd
import numpy as np
from sklearn.metrics import mean_absolute_error,mean_squared_error,r2_score
import mlflow
import joblib
from urllib.parse import urlparse
import mlflow.sklearn
import json

In [15]:
class ModelEvaluation():
    def __init__(self,config: ModelEvalutionConfig):
        self.config=config
    
    def eva_metrics(self,actual,pred):
        rmse=np.sqrt(mean_squared_error(actual,pred))
        mae=mean_absolute_error(actual,pred)
        r2=r2_score(actual,pred)

        return rmse,mae,r2
    

    def log_into_mlflow(self):

        test_data=pd.read_csv(self.config.test_datapath)

        model=joblib.load(self.config.model_filepath)

        test_x=test_data.drop(self.config.target_column,axis=1)

        test_y=test_data[self.config.target_column]

        mlflow.set_registry_uri(self.config.mlflow_uri)

        type_of_uri=urlparse(mlflow.get_tracking_uri()).scheme

       
        with mlflow.start_run():

            predicted_data=model.predict(test_x)

            (rmse,mae,r2)=self.eva_metrics(test_y,predicted_data)

            scores={"rmse":rmse,"mae":mae,"r2":r2}
            save_json(path=Path(self.config.metrics_file_name),data=scores)
            
            for key, value in self.config.all_params.items():
              mlflow.log_param(key, value)

            mlflow.log_param("params",self.config.all_params)

            mlflow.log_metric("rmse",rmse)
            mlflow.log_metric("mae",mae)
            mlflow.log_metric("r2",r2)


            if type_of_uri != "file":

                mlflow.sklearn.log_model(model,"model",registered_model_name="ElasticNet")
            else:
                mlflow.sklearn.log_model(model,"model")
          

In [16]:
try:
    config=ConfigurationManager()
    evaluation_data=config.get_model_evaluation_data()
    evaluation=ModelEvaluation(config=evaluation_data)
    evaluation.log_into_mlflow()
except Exception as e:
    raise e

[2025-01-10 23:00:58,741: INFO: common: yaml file  from these path config\config.yaml successfully loaded]
[2025-01-10 23:00:58,741: INFO: common: yaml file  from these path params.yaml successfully loaded]
[2025-01-10 23:00:58,753: INFO: common: yaml file  from these path schema.yaml successfully loaded]
[2025-01-10 23:00:58,754: INFO: common: successfully created directory at :artifacts]
[2025-01-10 23:00:58,755: INFO: common: successfully created directory at :artifacts/model_evaluation]
[2025-01-10 23:00:59,623: INFO: common: converted to json artifacts\model_evaluation\metrics.json]


2025/01/10 23:01:11 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
Successfully registered model 'ElasticNet'.
2025/01/10 23:01:19 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: ElasticNet, version 1
Created version '1' of model 'ElasticNet'.


🏃 View run popular-snail-744 at: https://dagshub.com/YagantiAshok177/datascienceproject.mlflow/#/experiments/0/runs/6e5e569c9b4849e28c63df2e06bbc7c3
🧪 View experiment at: https://dagshub.com/YagantiAshok177/datascienceproject.mlflow/#/experiments/0
